# Parcial de Regresión Lineal


## Preguntas de Negocio
1. **Predicción:** Si llega un carro con 92670.5 km, ¿a cuánto debería venderlo según el modelo?
2. **Inversión:** Como inversionista, ¿cuáles son los vehículos (según su kilometraje) que podrían generar un margen de ganancia atractivo?


In [3]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

%matplotlib inline

## 1. Cargar y Preprocesar Datos

Se descarga el dataset de vehículos, se descomprime y se carga en un DataFrame. Además, se eliminan los valores faltantes.

In [16]:
# Descargar y descomprimir el dataset
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/cars_dataset.zip
!unzip -o cars_dataset.zip


--2025-03-12 20:52:59--  https://github.com/javierherrera1996/lecture_analytics/raw/main/cars_dataset.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/cars_dataset.zip [following]
--2025-03-12 20:53:00--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/cars_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19753181 (19M) [application/zip]
Saving to: ‘cars_dataset.zip.2’

cars_dataset.zip.2  100%[===================>]  18.84M  --.-KB/s    in 0.06s   

2025-03-12 20:53:00 (326 MB/s) - ‘cars_dataset.zip.2’ saved [19753

In [6]:
df=pd.read_csv('car_prices.csv')
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


## 2. Análisis Exploratorio de Datos (EDA)

Se generan estadísticas descriptivas para analizar las variables numéricas y entender la distribución de los datos.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558799 non-null  float64
 14  sellingprice  558825 non-null  float64
 15  saledate      558825 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 68.2+ MB


In [12]:
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [18]:
print(df.head())
print(df.columns)

   year   make                model        trim   body transmission  \
0  2015    Kia              Sorento          LX    SUV    automatic   
1  2015    Kia              Sorento          LX    SUV    automatic   
2  2014    BMW             3 Series  328i SULEV  Sedan    automatic   
3  2015  Volvo                  S60          T5  Sedan    automatic   
4  2014    BMW  6 Series Gran Coupe        650i  Sedan    automatic   

                 vin state  condition  odometer  color interior  \
0  5xyktca69fg566472    ca        5.0   16639.0  white    black   
1  5xyktca69fg561319    ca        5.0    9393.0  white    beige   
2  wba3c1c51ek116351    ca       45.0    1331.0   gray    black   
3  yv1612tb4f1310987    ca       41.0   14282.0  white    black   
4  wba6b2c57ed129731    ca       43.0    2641.0   gray    black   

                                   seller      mmr  sellingprice  \
0                 kia motors america  inc  20500.0       21500.0   
1                 kia motors ameri

In [19]:
print("Estadísticas descriptivas:")
print(df.describe())

Estadísticas descriptivas:
                year      condition       odometer            mmr  \
count  472325.000000  472325.000000  472325.000000  472325.000000   
mean     2010.210980      30.774177   66701.732040   13837.058964   
std         3.822151      13.286866   51939.586894    9532.229273   
min      1990.000000       1.000000       1.000000      25.000000   
25%      2008.000000      24.000000   28137.000000    7425.000000   
50%      2012.000000      35.000000   51085.000000   12300.000000   
75%      2013.000000      41.000000   96590.000000   18300.000000   
max      2015.000000      49.000000  999999.000000  182000.000000   

        sellingprice  
count  472325.000000  
mean    13690.512058  
std      9613.033738  
min         1.000000  
25%      7200.000000  
50%     12200.000000  
75%     18200.000000  
max    230000.000000  


## 3. Modelo de Regresión Lineal

Se utiliza la variable 'odometer' para predecir 'sellingprice'. Se dividen los datos en conjunto de entrenamiento y prueba (80%-20%).

In [30]:

X = df[['odometer']]
y = df['sellingprice']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


modelo = LinearRegression()
modelo.fit(X_train, y_train)


print("Coeficiente:", modelo.coef_)
print("Intersección:", modelo.intercept_)


Coeficiente: [-0.10704423]
Intersección: 20836.521358799273


## 4. Evaluación del Modelo

Se evalúa el desempeño del modelo en el conjunto de prueba utilizando métricas como MSE, MAE y R².

In [36]:
y_pred = modelo.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R²:", r2)



MSE: 60380220.39908442
MAE: 5399.032012605542
R²: 0.3356777870212565


## 5. Predicción y Respuestas a Preguntas de Negocio

### Pregunta 1: Precio Estimado para un Carro con 92670.5 km

Utilizamos el modelo para predecir el precio de venta de un carro con 92670.5 km.

In [38]:
nuevo_odometro = pd.DataFrame({'odometer': [92670.5]})
precio_estimado = modelo.predict(nuevo_odometro)
print("Precio estimado para un carro con 92670.5 km:", precio_estimado[0])

Precio estimado para un carro con 92670.5 km: 10916.679128133772


## 6. Análisis Crítico de Resultados (10 Puntos Cerrados)

Responde a las siguientes preguntas de manera cerrada (Sí/No). Estas preguntas facilitan una calificación rápida:

1. ¿El coeficiente negativo (-0.107) indica que a mayor kilometraje, menor precio de venta? (Sí/No)  SI
2. ¿Un R² de 0.336 sugiere que el modelo explica más del 50% de la variabilidad en el precio de venta? (Sí/No) NO
3. ¿El valor elevado del MSE indica que existen errores importantes en las predicciones? (Sí/No) SI
4. ¿El MAE obtenido es suficientemente bajo para garantizar predicciones precisas en todos los casos? (Sí/No) NO
5. ¿La eliminación de filas con valores faltantes puede reducir la robustez del modelo? (Sí/No) SI
6. ¿Incluir variables adicionales como año, condición y marca podría mejorar la capacidad predictiva del modelo? (Sí/No) SI
7. ¿La relación negativa entre kilometraje y precio es consistente con el comportamiento esperado en el mercado de vehículos? (Sí/No) SI
8. ¿Dividir los datos en 80% entrenamiento y 20% prueba es una práctica recomendada para evaluar el modelo? (Sí/No) SI
